<a href="https://colab.research.google.com/github/BastineJohns/100DaysOfML/blob/master/Resume_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy-transformers
!pip install -U spacy
!pip install spacy_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 1.1 MB 15.4 MB/s 
     |████████████████████████████████| 4.7 MB 51.1 MB/s 
     |████████████████████████████████| 6.6 MB 55.1 MB/s 
     |████████████████████████████████| 163 kB 45.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

# New Section

In [3]:
!nvidia-smi

Thu Oct 13 10:15:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W /  70W |    336MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:

!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 82 (delta 16), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (82/82), done.


In [5]:
cv_data = json.load(open(r'/content/CV-Parsing-using-Spacy-3/data/training/train_data.json'))


In [6]:
len(cv_data)

200

In [40]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annotations in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annotations['entities']
    print(annot)

    ents = []
    entity_indices = []

    for start, end, label in annot:
      print(start,end,label)
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue    
      
      entity_indices = entity_indices + list(range(start, end))
      
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
        
      except:
        continue


      if span is None:
        err_data = str([start, end]) + " " + str(text) + "\n"
        file.write(str(err_data.encode('utf-8'))) 
      
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc) 
    except:
      pass

  return db   




    

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)
len(train)



140

In [10]:
file = open('error.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)

db.to_disk('test_data.spacy')

file.close()

 10%|█         | 14/140 [00:00<00:01, 67.60it/s]

[[288, 293, 'Location'], [267, 284, 'College Name'], [250, 265, 'Degree'], [211, 216, 'Location'], [190, 207, 'College Name'], [177, 188, 'Degree'], [152, 157, 'Location'], [139, 148, 'Companies worked at'], [118, 137, 'Designation'], [55, 99, 'Email Address'], [15, 20, 'Location'], [0, 14, 'Name']]
288 293 Location
267 284 College Name
250 265 Degree
211 216 Location
190 207 College Name
177 188 Degree
152 157 Location
139 148 Companies worked at
118 137 Designation
55 99 Email Address
15 20 Location
0 14 Name
[[1439, 1468, 'College Name'], [1434, 1437, 'Degree'], [1371, 1376, 'Degree'], [186, 195, 'Location'], [155, 171, 'Designation'], [89, 136, 'Email Address'], [35, 44, 'Location'], [17, 33, 'Designation'], [0, 16, 'Name']]
1439 1468 College Name
1434 1437 Degree
1371 1376 Degree
186 195 Location
155 171 Designation
89 136 Email Address
35 44 Location
17 33 Designation
0 16 Name
[[3758, 3764, 'Companies worked at'], [3630, 3636, 'Companies worked at'], [3532, 3538, 'Companies work

 15%|█▌        | 21/140 [00:00<00:02, 52.72it/s]

[[210, 251, 'Skills'], [189, 200, 'College Name'], [165, 187, 'Degree'], [117, 126, 'Companies worked at'], [104, 115, 'Designation'], [43, 85, 'Email Address'], [13, 20, 'Location'], [0, 12, 'Name']]
210 251 Skills
189 200 College Name
165 187 Degree
117 126 Companies worked at
104 115 Designation
43 85 Email Address
13 20 Location
0 12 Name
[[2076, 2079, 'Skills'], [2058, 2091, 'Skills'], [2058, 2074, 'Skills'], [1972, 1981, 'Skills'], [1935, 1951, 'Skills'], [1911, 1914, 'Skills'], [1865, 1868, 'Skills'], [1862, 1890, 'Skills'], [1818, 1841, 'Skills'], [1750, 1782, 'College Name'], [1744, 1748, 'Graduation Year'], [1711, 1742, 'College Name'], [1499, 1503, 'Graduation Year'], [1419, 1489, 'College Name'], [1374, 1417, 'Degree'], [447, 467, 'Companies worked at'], [430, 445, 'Designation'], [32, 41, 'Location'], [15, 30, 'Designation'], [0, 14, 'Name']]
2076 2079 Skills
2058 2091 Skills
2058 2074 Skills
1972 1981 Skills
1935 1951 Skills
1911 1914 Skills
1865 1868 Skills
1862 1890 Ski

 19%|█▉        | 27/140 [00:00<00:02, 39.94it/s]

...


5333 5337 Graduation Year
5240 5282 College Name
5171 5238 Degree
3426 3441 Companies worked at
3385 3389 Graduation Year
3371 3381 Location
3359 3369 Location
3340 3355 Companies worked at
1104 1108 Graduation Year
1082 1092 Location
1070 1080 Location
1051 1066 Companies worked at
1031 1049 Designation
887 891 Graduation Year
57 98 Email Address
24 34 Location
12 22 Location
0 11 Name
[[7858, 7886, 'College Name'], [7853, 7856, 'Degree'], [7823, 7851, 'College Name'], [7793, 7821, 'Degree'], [428, 454, 'Companies worked at'], [349, 379, 'Designation'], [73, 117, 'Email Address'], [15, 46, 'Designation'], [0, 14, 'Name']]
7858 7886 College Name
7853 7856 Degree
7823 7851 College Name
7793 7821 Degree
428 454 Companies worked at
349 379 Designation
73 117 Email Address
15 46 Designation
0 14 Name
[[2051, 2567, 'Skills'], [1914, 1943, 'College Name'], [1856, 1912, 'Degree'], [1734, 1767, 'Designation'], [1665, 1694, 'Companies worked at'], [1653, 1663, 'Designation'], [91, 133, 'Email 

 93%|█████████▎| 56/60 [00:01<00:00, 37.30it/s]

3271 College Name
3236 3240 Graduation Year
3207 3210 College Name
3172 3205 Degree
2399 2409 Degree
1754 1792 Email Address
1206 1212 Companies worked at
1032 1038 Companies worked at
1009 1030 Designation
224 234 Degree
124 134 Degree
81 119 Email Address
32 47 Location
0 8 Name
[[2707, 2711, 'Skills'], [2683, 2693, 'Skills'], [2677, 2681, 'Skills'], [2670, 2673, 'Skills'], [2662, 2666, 'Skills'], [2654, 2658, 'Skills'], [2626, 2629, 'Skills'], [2607, 2624, 'Skills'], [2599, 2605, 'Skills'], [2594, 2597, 'Skills'], [2585, 2592, 'Skills'], [2561, 2565, 'Skills'], [2541, 2556, 'Skills'], [2515, 2521, 'Skills'], [2499, 2503, 'Skills'], [2487, 2497, 'Skills'], [2456, 2465, 'Skills'], [2386, 2390, 'Graduation Year'], [2341, 2376, 'College Name'], [2333, 2339, 'Degree'], [1982, 1986, 'Skills'], [1977, 1980, 'Skills'], [1971, 1975, 'Skills'], [1959, 1969, 'Skills'], [1888, 1904, 'Designation'], [1824, 1840, 'Designation'], [1436, 1440, 'Skills'], [1085, 1125, 'Email Address'], [911, 915, 'S

100%|██████████| 60/60 [00:01<00:00, 41.27it/s]


3247 College Name
3175 3223 Degree
2272 2316 Email Address
934 943 Location
908 930 Companies worked at
766 775 Location
740 762 Companies worked at
58 102 Email Address
15 24 Location
0 14 Name
[[311, 331, 'Skills'], [265, 292, 'College Name'], [249, 263, 'Degree'], [171, 189, 'Companies worked at'], [158, 169, 'Designation'], [85, 139, 'Email Address'], [43, 54, 'Location'], [0, 23, 'Name']]
311 331 Skills
265 292 College Name
249 263 Degree
171 189 Companies worked at
158 169 Designation
85 139 Email Address
43 54 Location
0 23 Name


# New Section

In [11]:
len(db.tokens)

60

In [41]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train /content/train_data.spacy --paths.dev /content/test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-10-13 10:55:55,600] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-10-13 10:55:55,610] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-10-13 10:55:55,614] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-10-13 10:55:55,615] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrain

In [17]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 8.2 MB/s 


Model Test


In [42]:
nlp = spacy.load('/content/output/model-best')


In [15]:
!pip install fitz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.2 MB 8.2 MB/s 
     |████████████████████████████████| 95 kB 4.7 MB/s 
     |████████████████████████████████| 130 kB 69.4 MB/s 
     |████████████████████████████████| 421 kB 68.2 MB/s 
     |████████████████████████████████| 500 kB 64.6 MB/s 
     |████████████████████████████████| 5.1 MB 56.7 MB/s 
     |████████████████████████████████| 41 kB 784 kB/s 
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34547 sha256=6998cf0f97ffd9a3f1783939dc937bcfba4da80df583cf49aff23ae3625d04e5
  Stored in directory: /root/.cache/pip/wheels/0d/c4/19/13d74440f2a571841db6b6e0a273694327498884dafb9cf978
Successfully built configobj


In [71]:
from pdfminer.high_level import extract_text

In [73]:
fname = '/content/Beige Minimalist Social Media Manager Resume (1).pdf'
doc = extract_text(fname)

In [74]:
doc

'MARIZA TOMS\n\nUNDERGRADUATE H.BSc STUDENT\n\nCONTACT\n\nPhone :\n\n+1 807-708-6789\n\nEmail  :\n\nmarizatoms12@gmail.com\n\nAddress :\n\n210 Leland Avenue South\n\nThunder Bay, ON\n\nP7E 2N5\n\nEDUCATION\n\n2022-2026\n\nHBSc BIOLOGY\n\nLakehead University,\n\nThunder Bay\n\n2019-2021\n\nBio-Math \n\nAKJM Higher Secondary School,\n\nIndia\n\nSKILLS\n\nMicrosoft Office (Powerpoint\n\n,Word and Excel)\n\nVerbal and Written\n\ncommunication skills\n\nCAREER OBJECTIVE\n\nTo secure a challenging position in a reputable organization \n\nto expand my learning , knowledge and skills.\n\nWORK EXPERIENCE\n\n2021-2022\n\nCashier\n\nBilling clerk\n\nAswas Community Pharmacy, India\n\nAVAILABILITY\n\nMONDAY        8.00AM   -  2.00 PM\nTUESDAY        12.00PM  -  5.00 PM\nWEDNESDAY   8.00AM   -  2.00 PM\nTHURSDAY      8.00 AM  -  1.00  PM\nFRIDAY           6.00 PM   - 10.00PM\nSATURDAY      FULLY AVAILABLE\nSUNDAY         FULLY  AVAILABLE\n\nDECLARATION\n\nI hereby declare that this\n\nResume is tru

In [75]:
doc = str(doc)
text = doc.strip()
text = ' '.join(text.split())




In [76]:
text

'MARIZA TOMS UNDERGRADUATE H.BSc STUDENT CONTACT Phone : +1 807-708-6789 Email : marizatoms12@gmail.com Address : 210 Leland Avenue South Thunder Bay, ON P7E 2N5 EDUCATION 2022-2026 HBSc BIOLOGY Lakehead University, Thunder Bay 2019-2021 Bio-Math AKJM Higher Secondary School, India SKILLS Microsoft Office (Powerpoint ,Word and Excel) Verbal and Written communication skills CAREER OBJECTIVE To secure a challenging position in a reputable organization to expand my learning , knowledge and skills. WORK EXPERIENCE 2021-2022 Cashier Billing clerk Aswas Community Pharmacy, India AVAILABILITY MONDAY 8.00AM - 2.00 PM TUESDAY 12.00PM - 5.00 PM WEDNESDAY 8.00AM - 2.00 PM THURSDAY 8.00 AM - 1.00 PM FRIDAY 6.00 PM - 10.00PM SATURDAY FULLY AVAILABLE SUNDAY FULLY AVAILABLE DECLARATION I hereby declare that this Resume is true and correct to the best of my knowledge. Good observation and listening DATE: skills . LOCATION : Leland Ave S Time management abilities. THUNDERBAY,ON Good team player with cr

In [63]:
!pip install pdfminer.six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 7.1 MB/s 
     |████████████████████████████████| 4.0 MB 53.5 MB/s 


In [77]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text,"---",ent.label_)

MARIZA TOMS --- Name
H.BSc STUDENT --- Degree
Microsoft Office (Powerpoint ,Word and Excel) Verbal and Written communication skills CAREER OBJECTIVE To secure a challenging position in a reputable organization to expand my learning , knowledge and skills. --- Skills


In [ ]:
doc = nlp("Bastine Johns has good coomunicative skills and programming skills")

In [ ]:
doc

Bastine Johns has good coomunicative skills and programming skills

In [ ]:
doc.ents

(Bastine Johns,)

In [ ]:
nlp